In [2]:
import pandas as pd
import os
import numpy as np
data_folder = "GetNo1/"


raw_data_path = os.path.join(data_folder, 'carSales.csv') 
insight_data_path = os.path.join(data_folder, 'GetNo1ResFormatted.csv') 
raw_data_df = pd.read_csv(raw_data_path)
insight_df = pd.read_csv(insight_data_path)
# Process the subspace features

sub_space_col = list(insight_df['Subspace'])
array = []
for idx, raw in enumerate(sub_space_col):
    array.append(raw.split(';')[0:-1])
array = np.array(array)

#  hard code
feature_columns = ['Year', 'Brand', 'Category']
#  hard code finished ----------

for idx, c in enumerate(feature_columns):
    insight_df[c] = array[:,idx]

current_columns = feature_columns + [c for c in insight_df.columns if c not in feature_columns + ['Subspace']]
insight_df = insight_df[current_columns]
insight_df['id'] = insight_df.index

def get_subspace_condition(subspace_dict, feature_columns, df):
    condition = pd.Series(True, index=df.index)
    for feature in feature_columns:
        if subspace_dict[feature] == '*':
            continue
        condition = condition & (df[feature] == subspace_dict[feature])
    return condition

def get_subspace_df(subspace_dict, feature_columns, df):
    condition = get_subspace_condition(subspace_dict, feature_columns, df)
    return df[condition]

def calc_coverage(row):
    sub_space_df = get_subspace_df(dict(row), feature_columns, raw_data_df)
    return len(sub_space_df.index.intersection(raw_data_df.index)) / len(sub_space_df.index.union(raw_data_df.index))

def calc_index_jaccard(df1, df2):
    return len(df1.index.intersection(df2.index)) / len(df1.index.union(df2.index))

insight_df['impact'] = insight_df.apply(lambda row: calc_coverage(row), axis=1)
# ---------
nodes, edges = [], []
for index, insight_row in insight_df.iterrows():
    nodes.append(dict(insight_row))
for i in range(0, len(nodes)):
    src = nodes[i]
    src_subspace_df = get_subspace_df(src, feature_columns, raw_data_df)
    for j in range(i+1, len(nodes)):
        dst = nodes[j]
        dst_subspace_df = get_subspace_df(dst, feature_columns, raw_data_df)
        edges.append({'source': src['id'], 'target':dst['id'],'jaccard_index': calc_index_jaccard(src_subspace_df, dst_subspace_df)})
    print(i)
print('Finish')

graph = {
    'nodes': nodes,
    'edges': edges,
    'featrues': feature_columns
}

import json
with open('insight_graph.json', 'w') as output_file:
    json.dump(graph, output_file)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
Finish


In [3]:
insight_df

,Year,Brand,Category,Breakdown,Measure,Max Avg Value,Breakdown Value,id,impact
0,*,*,*,Year,Sales,91726.56364,2007/1/1,0,1.000000
1,*,*,*,Brand,Sales,177257.04000,Honda,1,1.000000
2,*,*,*,Category,Sales,149148.97440,Midsize,2,1.000000
3,2007/1/1,*,*,Brand,Sales,206827.20000,Honda,3,0.200000
4,2007/1/1,*,*,Category,Sales,179617.42860,Midsize,4,0.200000
5,2008/1/1,*,*,Brand,Sales,203119.60000,Honda,5,0.200000
6,2008/1/1,*,*,Category,Sales,150622.00000,Midsize,6,0.200000
7,2009/1/1,*,*,Brand,Sales,164125.40000,Honda,7,0.200000
8,2009/1/1,*,*,Category,Sales,131875.00000,Midsize,8,0.200000
9,2010/1/1,*,*,Brand,Sales,163012.60000,Honda,9,0.200000
